In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from communfunctions import *
from PIL import Image
from sklearn.utils import shuffle
import pickle


#models
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

#preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score



In [3]:
#constants
IMG_SIZE = 32

In [7]:
def extract_hog_features(img):
    """
    Extracts Histogram of Oriented Gradients (HOG) features from an image.

    Parameters:
    img (numpy.ndarray): A numpy array of shape (H, W) representing the input image.

    Returns:
    numpy.ndarray: A numpy array of shape (N,) containing the HOG features.
    """
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    win_size = (IMG_SIZE, IMG_SIZE)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9 
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [8]:
def load_dataset(path_to_dataset):
    features = []
    labels = []
    img_path=[]
    labels_folders = os.listdir(path_to_dataset)

    for label in labels_folders:
        path = os.path.join(path_to_dataset, label)
        img_filenames = os.listdir(path)
        for fn in img_filenames:
            labels.append(label)

            path = os.path.join(path_to_dataset, label, fn)
            img = cv2.imread(path)
            features.append(extract_hog_features(img))
            img_path.append(path)
        
    return features, labels, img_path 


In [9]:
def negative_transformation(img):
    """
    This function takes an image and returns a negative image

    Parameters:
    img: np.array
        The image to be processed
    
    Returns:
    negative_img: np.array
        The negative image
    """
    return 255-img

def aug(path_to_dataset):
    labels_folders = os.listdir(path_to_dataset)

    for label in labels_folders:
        path = os.path.join(path_to_dataset, label)
        img_filenames = os.listdir(path)
        for fn in img_filenames:
            if fn.split('.')[-1] != 'jpg' or fn.split('.')[-1] != 'jpeg':
                continue

            path = os.path.join(path_to_dataset, label, fn)
            img = cv2.imread(path)
            neg_image = negative_transformation(img)
            neg_image_path = path.split('.')[0]+'_neg.jpg'
            cv2.imwrite(neg_image_path,neg_image)
        

In [10]:
def convert_to_jpg(path):
    labels_folders = os.listdir(path)
    for label in labels_folders:
        print(label)
        label_path = os.path.join(path, label)
        if not os.path.isdir(label_path): 
            continue
        for img in os.listdir(label_path):
            img_path = os.path.join(label_path, img)
            
            if img.split('.')[-1].lower() == 'jpg':
                continue  
            
            png_image = Image.open(img_path)
            
            if png_image.mode == 'RGBA':
                white_bg = Image.new('RGB', png_image.size, (255, 255, 255))
                white_bg.paste(png_image, (0, 0), png_image)
                
                jpg_path = os.path.join(label_path, img.split('.')[0] + '.jpg')
                white_bg.save(jpg_path)
            else:
                jpg_path = os.path.join(label_path, img.split('.')[0] + '.jpg')
                png_image = png_image.convert('RGB')
                png_image.save(jpg_path)

            os.remove(img_path)


In [11]:
x_data_handwritten, y_data_handwritten ,img_paths1= load_dataset('handwritten')

In [12]:
x_data_digital, y_data_digital ,img_paths2= load_dataset('digital')

In [13]:
x_data_digital2, y_data_digital2 ,img_paths3= load_dataset('printed-digits-dataset')

In [15]:
x_data_sudoku, y_data_sudoku ,img_paths4= load_dataset('sudoku_dataset')

In [16]:
df_hand=pd.DataFrame(x_data_handwritten)
df_hand['label']=y_data_handwritten
df_hand['img_path']=img_paths1

In [17]:
df_digital=pd.DataFrame(x_data_digital)
df_digital['label']=y_data_digital
df_digital['img_path']=img_paths2

In [18]:
df_digital2=pd.DataFrame(x_data_digital2)
df_digital2['label']=y_data_digital2
df_digital2['img_path']=img_paths3

In [19]:
df_so=pd.DataFrame(x_data_sudoku)
df_so['label']=y_data_sudoku
df_so['img_path']=img_paths4

In [20]:
df_hand.groupby('label').size()

label
1    1201
2    1201
3    1201
4    1201
5    1201
6    1201
7    1201
8    1201
9    1201
dtype: int64

In [21]:
df_digital.groupby('label').size()

label
1    573
2    573
3    573
4    573
5    573
6    573
7    573
8    573
9    573
dtype: int64

In [22]:
df_digital2.groupby('label').size()

label
1    742
2    750
3    605
4    630
5    592
6    526
7    688
8    560
9    565
dtype: int64

In [23]:
df_so.groupby('label').size()

label
1    451
2    419
3    443
4    455
5    464
6    469
7    465
8    466
9    429
dtype: int64

In [24]:
digital=pd.concat([df_digital,df_digital2,df_so])
digital.groupby('label').size()

label
1    1766
2    1742
3    1621
4    1658
5    1629
6    1568
7    1726
8    1599
9    1567
dtype: int64

In [28]:
df_digital.groupby('label').size()

label
1    1766
2    1742
3    1621
4    1658
5    1629
6    1568
7    1726
8    1599
9    1567
dtype: int64

In [29]:
df_hand.drop(columns=['img_path'],inplace=True)
X_hand = df_hand.drop(columns=['label']) 
y_hand = df_hand['label']

# First, split the data into 90% training/validation and 10% test
# Then, split the 90% training/validation into 80% training and 15% validation

X_train_val, X_test_hand, y_train_val, y_test_hand = train_test_split(X_hand, y_hand, test_size=0.05, random_state=42)
X_train_hand, X_val_hand, y_train_hand, y_val_hand = train_test_split(X_train_val, y_train_val, test_size=0.1667, random_state=42)


In [30]:
df_digital.drop(columns=['img_path'],inplace=True)
X_digital = df_digital.drop(columns=['label']) 
y_digital = df_digital['label']

# First, split the data into 90% training/validation and 10% test
# Then, split the 90% training/validation into 80% training and 15% validation

X_train_val, X_test_digital, y_train_val, y_test_digital = train_test_split(X_digital, y_digital, test_size=0.05, random_state=42)
X_train_digital, X_val_digital, y_train_digital, y_val_digital = train_test_split(X_train_val, y_train_val, test_size=0.1667, random_state=42)


In [31]:
X_train = pd.concat([X_train_hand, X_train_digital], ignore_index=True)
y_train = pd.concat([y_train_hand, y_train_digital], ignore_index=True)

In [32]:
x_cv = pd.concat([X_val_hand, X_val_digital], ignore_index=True)
y_cv = pd.concat([y_val_hand, y_val_digital], ignore_index=True)

In [33]:
x_test = pd.concat([X_test_hand, X_test_digital], ignore_index=True)
y_test = pd.concat([y_test_hand, y_test_digital], ignore_index=True)

In [34]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.289801,0.075783,0.000000,0.166717,0.030005,0.289801,0.035834,0.057322,0.026176,0.040715
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.129441,0.248459,0.013911,0.000000,0.000000,0.022811,0.033601,0.072784,0.031865,0.177146
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.095943,0.000000,0.001696,0.005074,0.000000,0.020895,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.187030,0.009437,0.003658,0.013192,0.034472,0.083031,0.001586,0.000000,0.010443,0.004343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20327,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.257484,0.054814,0.021716,0.064983,0.000000,0.109629,0.000000,0.000000,0.000000,0.054814
20328,0.020984,0.008239,0.025692,0.016286,0.056809,0.030738,0.037135,0.016790,0.036689,0.115768,...,0.000209,0.014726,0.153808,0.350676,0.102845,0.022379,0.012383,0.000000,0.000000,0.009388
20329,0.012588,0.105946,0.058844,0.010146,0.012327,0.002732,0.009363,0.010538,0.007592,0.239322,...,0.002486,0.007316,0.271282,0.356753,0.163798,0.014353,0.001820,0.000890,0.000090,0.002771
20330,0.053611,0.192529,0.196544,0.037601,0.311895,0.013061,0.010073,0.106359,0.057066,0.045904,...,0.027929,0.149280,0.289130,0.174884,0.137889,0.289130,0.064262,0.199904,0.063895,0.118384


In [35]:
y_train.value_counts().sort_index()

label
1    2332
2    2321
3    2266
4    2277
5    2249
6    2165
7    2357
8    2172
9    2193
Name: count, dtype: int64

In [36]:

X_train, y_train = shuffle(X_train, y_train, random_state=42)
x_cv, y_cv = shuffle(x_cv, y_cv, random_state=42)
x_test, y_test = shuffle(x_test, y_test, random_state=42)


In [37]:
param_grid = {'n_neighbors': [5,7,8]}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_knn = grid_search.best_estimator_

y_pred = best_knn.predict(x_cv)

accuracy = accuracy_score(y_cv, y_pred)
print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy:.2f}")


Best Parameters: {'n_neighbors': 5}
Accuracy: 0.94


In [38]:
predictions = best_knn.predict(x_test)
accuracy = accuracy_score(y_test, predictions)

In [39]:
accuracy

0.9330739299610895

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

param_grid = {
    'criterion': ['gini', 'entropy'],  # Splitting criteria
    'max_depth': [3, 5, 10, None],     # Tree depth
}

decision_tree = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_tree = grid_search.best_estimator_

y_pred = best_tree.predict(x_cv)

accuracy = accuracy_score(y_cv, y_pred)
print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy:.2f}")


In [ ]:
predictions = best_tree.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

In [142]:
import pickle

with open('knn_model5.pkl', 'wb') as file:
    pickle.dump(best_knn, file)

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],     
    'max_depth': [3, 5, 10],       
    'learning_rate': [0.01, 0.1, 0.2],  
}

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_xgb = grid_search.best_estimator_

y_pred = best_xgb.predict(x_cv)

accuracy = accuracy_score(y_cv, y_pred)
print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy:.2f}")


In [ ]:
predictions = best_xgb.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
accuracy